In [1]:
# Identificação facial

In [2]:
import os
import numpy as np
import cv2
import face_recognition

In [3]:
diretorio_trabalho = '/home/pi/Documents/Identificação facial/Indivíduos/'
os.chdir(diretorio_trabalho)

In [4]:
class Pessoa:
    def __init__(self, nome='Não definido'):
        self.nome = nome
        self.minuncias = []

    def calculaMinuncia(self, endereco_imagem):  
        imagem = face_recognition.load_image_file(endereco_imagem)
        minuncia = face_recognition.face_encodings(imagem)[0]
        self.minuncias.append(minuncia)     
    
    def imagemMatch(self, face_minuncias):
        dist = 0
        for aux_minuncias in self.minuncias:
            dist = dist + np.linalg.norm(face_minuncias-aux_minuncias)
        dist = dist/len(self.minuncias)      
        return dist

In [5]:
# Cria "Jaime".
jaime = Pessoa('Jaime')
jaime.calculaMinuncia(diretorio_trabalho+'Jaime/WhatsApp Image 2021-02-10 at 10.59.35.jpeg')
jaime.calculaMinuncia(diretorio_trabalho+'Jaime/WhatsApp Image 2021-02-10 at 10.59.36.jpeg')
jaime.calculaMinuncia(diretorio_trabalho+'Jaime/WhatsApp Image 2021-02-10 at 10.59.36 (1).jpeg')
jaime.calculaMinuncia(diretorio_trabalho+'Jaime/WhatsApp Image 2021-02-10 at 10.59.37.jpeg')
jaime.calculaMinuncia(diretorio_trabalho+'Jaime/WhatsApp Image 2021-02-10 at 10.59.37 (1).jpeg')

In [6]:
# Cria "Obama".
obama = Pessoa('Obama')
obama.calculaMinuncia(diretorio_trabalho+'Obama/obama.jpg')

In [20]:
# Cria "Priscila".
priscila = Pessoa('Priscila')
priscila.calculaMinuncia(diretorio_trabalho+'Priscila/WhatsApp Image 2021-02-10 at 10.58.17.jpeg')
priscila.calculaMinuncia(diretorio_trabalho+'Priscila/WhatsApp Image 2021-02-10 at 10.58.18.jpeg')
priscila.calculaMinuncia(diretorio_trabalho+'Priscila/WhatsApp Image 2021-02-10 at 10.58.18 (1).jpeg')
priscila.calculaMinuncia(diretorio_trabalho+'Priscila/WhatsApp Image 2021-02-10 at 10.58.19.jpeg')

In [16]:
# Calcula as minuncias de uma imagem qualquer.
endereco_imagem = diretorio_trabalho+'Obama2.jpg'
imagem = face_recognition.load_image_file(endereco_imagem)
faces_locacoes = face_recognition.face_locations(imagem)
face_minuncias = None
n_faces = len(faces_locacoes)
if n_faces==1:
    print("Face detectada.")
    face_minuncias = face_recognition.face_encodings(imagem)[0]
elif n_faces==0:
    print("Nenhuma face detectada!")
else:
    print("Mais de uma face detectada!")       

Face detectada.


In [17]:
# Obtem o match simples.
print(obama.imagemMatch(face_minuncias))

0.41545687830585304


In [21]:
print(jaime.imagemMatch(face_minuncias))

0.7312607231844565


In [22]:
print(priscila.imagemMatch(face_minuncias))

0.8087826078959571


In [23]:
class grupoPessoas:
    def __init__(self, nome='Não definido'):
        self.nome = nome
        self.pessoas = []
    
    def inserePessoa(self, pessoa):
        self.pessoas.append(pessoa)
        
    def minuncias():
        minuncias = []
        nomes = []
        for aux_pessoa in self.pessoas:
            self.minuncias.append(aux_pessoa.minuncias[:])
            self.nomes.append([aux_pessoa.nome]*len(aux_pessoa))
        return {'minuncias': minuncias, 'nomes': nomes}
    
    def calibraClassificador(self):
        pass
    